# k nearest neighbor
### import libs

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors, KNeighborsRegressor
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error

### bereinigen

In [2]:
def price_to_float(price):
    price = float(price.replace('$','').replace(',',''))
    return price

In [3]:
listings = pd.read_csv('~/studies/analytics/listings.csv')
original = listings.copy()
listings.dropna()
listings.shape
listings['price'] = listings['price'].map(price_to_float)
listings['price']
df = listings[['beds', 'bedrooms', 'room_type', 'price']]
df.dropna()

,beds,bedrooms,room_type,price
0,1.0,1.0,Private room,21.0
1,2.0,1.0,Entire home/apt,60.0
2,2.0,1.0,Entire home/apt,90.0
3,1.0,1.0,Private room,28.0
4,1.0,1.0,Entire home/apt,125.0
...,...,...,...,...
24417,1.0,1.0,Private room,54.0
24418,0.0,0.0,Entire home/apt,60.0
24419,1.0,0.0,Private room,36.0
24420,2.0,1.0,Private room,50.0


In [4]:
df = df[df['beds'].notna() == True]
df = df[df['bedrooms'].notna() == True]

In [5]:
df = pd.get_dummies(data=df, columns=['room_type'])

In [6]:
X = df.drop(columns=['price'])
y = df['price'].values

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [8]:
min_max_scaler = preprocessing.MinMaxScaler()
X_train_scaled = min_max_scaler.fit_transform(X_train)
X_test_scaled = min_max_scaler.fit_transform(X_test)

In [9]:
y_train_scaled = min_max_scaler.fit_transform(y_train.reshape(-1,1))
y_test_scaled = min_max_scaler.fit_transform(y_test.reshape(-1,1))

In [10]:
knn = KNeighborsRegressor(n_neighbors=1)

In [11]:
knn.fit(X_train_scaled, y_train_scaled)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                    weights='uniform')

In [12]:
y_pred = knn.predict(X_test_scaled)

In [13]:
mean_squared_error(y_test_scaled, y_pred)

0.0011492959707447184